In [3]:
import sys
import os
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

folder_path = '../src'
os.chdir(folder_path)

from utils.data_loader import (
    CloudDataset,
    visualize_batch,
    CloudTestDataset
)

from utils.augmentation import (
    get_albumentations_transform
)

from models.U_NET import (
    UNet,
)

from models.FPN_Net import (
    FPNNet
)

from utils.training_utils import (
    train_model,
    generate_confusion_matrix,
    compute_segmentation_metrics
)

from utils.visualization import (
    plot_metrics,
    plot_sample_prediction
)

In [4]:
images_dir = '../data/dados/images'
masks_dir = '../data/dados/masks'

all_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]

train_files, val_files = train_test_split(
    all_files,
    test_size=0.1,         # 10% dos dados para validação
    random_state=42
)

train_dataset = CloudDataset(images_dir, masks_dir, file_list=train_files)
val_dataset   = CloudDataset(images_dir, masks_dir, file_list=val_files)

train_loader  = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader    = DataLoader(val_dataset,   batch_size=16, shuffle=False)

print(f"Total de arquivos: {len(all_files)}")
print(f"Arquivos de treino: {len(train_files)}")
print(f"Arquivos de validação: {len(val_files)}")

Total de arquivos: 20331
Arquivos de treino: 18297
Arquivos de validação: 2034


In [5]:
from torchvision import transforms
test_images_dir = '../data/dados_teste/images'
test_files = [f for f in os.listdir(test_images_dir) if f.endswith('.png')]

# Cria dataset de teste que só retorna imagem:
test_dataset = CloudTestDataset(
    images_dir=test_images_dir,
    file_list=test_files,
    transform=None

)

print(f"Arquivos de teste: {len(test_files)}")

Arquivos de teste: 6730


# Treinamento sem aumento de dados

## U-Net

In [3]:
# Instanciar o modelo U-Net
model = UNet(in_channels=3, out_channels=1)
print(model)

# Treinar o modelo usando nossa função `train_model`
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=10,   
    lr=1e-4,
    device=None    
)

# Plotar métricas
plot_metrics(history)

# Gerar classification report no conjunto de validação
report = generate_classification_report(model, val_loader, device="cuda")
print("Classification Report (Val):\n", report)

UNet(
  (ups): ModuleList(
    (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
    (1): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (2): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
    (3): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(256, 256, kernel_size=(3, 3), st

Epoch [1/10] (Train):   0%|          | 2/4676 [01:19<51:25:53, 39.61s/it, acc=0.3938, iou=0.9621, loss=-0.5650]


KeyboardInterrupt: 

## FPN-Net

In [6]:
# Instanciar o modelo U-Net
model = FPNNet()

# Definir o caminho da pasta de checkpoints no Drive
checkpoint_dir = "/content/drive/MyDrive/pibic/meus_checkpoints/FPN-net"

# Garantir que a pasta exista
os.makedirs(checkpoint_dir, exist_ok=True)

c:\Users\arthu\anaconda3\envs\pibic_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\arthu\anaconda3\envs\pibic_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Treinamento Normal com Salvamento de Checkpoints e Melhor Modelo
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=20,
    lr=1e-4,
    device="cuda",
    checkpoint_dir=checkpoint_dir,
    save_best=True,
    metric_to_monitor="val_iou",
    mode="max"
)

# Plotar métricas
plot_metrics(history)

c:\Users\arthu\anaconda3\envs\pibic_env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
c:\Users\arthu\anaconda3\envs\pibic_env\lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch [1/20] (Train):   0%|          | 0/1144 [00:19<?, ?it/s]


KeyboardInterrupt: 

# Treinamento com aumento de dados

In [ ]:
# Definir transformações de aumento de dados com Albumentations
augmentation_transform = get_albumentations_transform(
    img_height=384,  
    img_width=384,
    rotation_limit=180.0,  # Rotação de -180 a +180 graus
    apply_horizontal_flip=True,
    horizontal_flip_prob=0.5,
    apply_color_jitter=True,
    brightness=0.2,
    contrast=0.2,
    saturation=0.2,
    hue=0.05,
    apply_gaussian_blur=True,
    gaussian_blur_kernel=3,
    gaussian_blur_prob=0.3,
    gaussian_blur_sigma=0.5
)

train_dataset = CloudDataset(images_dir, masks_dir, file_list=train_files, transform=augmentation_transform)
val_dataset   = CloudDataset(images_dir, masks_dir, file_list=val_files,   transform=augmentation_transform)

train_loader  = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader    = DataLoader(val_dataset,   batch_size=4, shuffle=False)

## U-Net